In [1]:
import pandas as pd

In [ ]:
#Reading csv file
#Data must be in numerical form, the output column must be named 'Class'.

In [9]:
df = pd.read_csv('train_20.csv')

In [10]:
df

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,Class
0,0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,0,232,8153,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,0,199,420,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25187,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
25188,0,334,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
25189,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
25190,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [ ]:
#In the function FLUX, the data must be preprocessed file and 'k' is the total number of features to be selected

In [11]:
def FLUX(data, k):    
    import numpy as np
    import pandas as pd
    
    # Move 'Class' to the first column
    first_column = data.pop('Class')
    data.insert(0, 'col0', first_column)
    
    n_rows = len(data)
    final_df = pd.DataFrame()
    
    # Function to create binary inverted columns
    def f_df(a):
        df1 = a.apply(lambda x: 1 - x)
        df1.columns = ['-' + col for col in a.columns]
        df4 = pd.concat([a, df1], axis=1)
        df4 = df4[[item for items in zip(a.columns, df1.columns) for item in items]]
        return df4

    Selected = []
    Rejected = []

    myheader = list(data.columns)

    # Iterate over each column except 'col0' (class column)
    for mycol in myheader[1:]:
        sortdata = data.sort_values(by=[mycol]).reset_index(drop=True)
        val_diff = sortdata[mycol].diff().fillna(0)

        # Find the unique cut points
        cut_mask = (val_diff != 0) & (
            ((sortdata['col0'].shift(1) != sortdata['col0']) & (sortdata['col0'].shift(-1) != sortdata['col0']))
        )
        cutpoints = (sortdata[mycol] + sortdata[mycol].shift(-1)) / 2
        cutpoints = cutpoints[cut_mask].dropna()

        if len(cutpoints) < 2000:
            Selected.append(mycol)
            binarydata = pd.DataFrame()
            
            for point in cutpoints:
                binarydata[f"{mycol}>={point}"] = (data[mycol] >= point).astype(int)

            # Calculate class counts
            N1 = data['col0'].sum()
            N0 = n_rows - N1

            binarydata.insert(len(binarydata.columns), 'classes', data['col0'])
            binarydata = binarydata.sort_values(by='classes', ascending=False).reset_index(drop=True)

            a = binarydata.pop('classes')
            binarydata = f_df(binarydata)
            binarydata.insert(len(binarydata.columns), 'classes', a)

            # Vectorized feature score calculations
            count_F1 = (binarydata.iloc[:N1].eq(binarydata['classes'], axis=0).mean())
            count_F0 = (binarydata.iloc[N1:].eq(binarydata['classes'], axis=0).mean())

            # Calculate feature scores
            count_FF = (2 * count_F1 * count_F0) / (count_F1 + count_F0).replace(0, np.nan).dropna()

            sorted_columns = count_FF.sort_values(ascending=False).index[:2]
            final_df = pd.concat([final_df, binarydata[sorted_columns]], axis=1)
        else:
            Rejected.append(mycol)

    # Remove duplicate columns
    final_df = final_df.loc[:, ~final_df.columns.duplicated()]

    # Final scoring on the selected features
    count_F11 = (final_df.iloc[:N1].eq(final_df['classes'], axis=0).mean())
    count_F00 = (final_df.iloc[N1:].eq(final_df['classes'], axis=0).mean())
    count_FF = (2 * count_F11 * count_F00) / (count_F11 + count_F00).replace(0, np.nan).dropna()

    sorted_columns = count_FF.sort_values(ascending=False).index[:151]
    final_df = final_df[sorted_columns]
    
    # Drop the 'classes' column
    final_df.drop(columns='classes', inplace=True)
    cl=final_df.columns
    #print(cl)
    cleaned_columns = [col.lstrip('-').split('>=')[0] for col in cl]
    unique_columns = list(dict.fromkeys(cleaned_columns))
    selected_feature = unique_columns[:k]
    return selected_feature


In [12]:
selected_features = FLUX(df,20)

In [13]:
selected_features

['dst_bytes',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'dst_host_diff_srv_rate',
 'count',
 'dst_host_serror_rate',
 'serror_rate',
 'dst_host_srv_serror_rate',
 'srv_serror_rate',
 'service_http',
 'dst_host_count',
 'srv_count',
 'dst_host_same_src_port_rate',
 'srv_diff_host_rate',
 'dst_host_rerror_rate',
 'src_bytes',
 'rerror_rate',
 'dst_host_srv_rerror_rate']